In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from pandas import json_normalize

from random import randint
from time import sleep

import requests

In [2]:
pd.set_option("display.max_columns", 0)

In [3]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}

for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

# 1. Biggest Playlist with all the Best Songs

In [4]:
playlist = sp.user_playlist_tracks("spotify", "54nv8jbrm4JoHEZ49Qvjgl")

In [5]:
# playlist["total"] 

In [6]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [7]:
all_tracks = get_playlist_tracks("54nv8jbrm4JoHEZ49Qvjgl")

In [8]:
# all_tracks

In [9]:
# type(all_tracks)

> * The name of the artist is inside a dictionary named "external_urls", that is inside a list which is the value of a key "artists", that is in the dictionary "album" which is the value for the key "track" at the main list. So the structure is as follows:
>
            ['track': {'album': {'artists': [{'name': 'Katy Perry'}]}}]
>
> Or:
>
            [{'track': {'artists': [{'name': 'Katy Perry'}]}}]
>
> * The name of the album is inside "album":
>
            ['track': {'album': {'name': 'Teenage Dream: The Complete Confection'}}]
>
> * And the name of the song:
>
            ['track': {'name': 'Firework'}}]
>
> This is the structure:
>
            [{
            'track':
                        {
                         'album': 
                                    {
                                    'artists': 
                                                [{
                                                'name': 'Katy Perry',
                                                }],
                                    'name': 'Teenage Dream: The Complete Confection',
                                    },
                        'artists': 
                                    [{
                                    'name': 'Katy Perry',
                                    }],
                        'name': 'Firework',
                        },
            }]
>
> So we´ll get a json normalised table with columns track.album.artists, track.artists; qhere we can extract the name of the artist, track.album.name; where we can extract the name of the album, and track.name, where we can extract the name of the song.

In [10]:
tracks = json_normalize(all_tracks)

In [11]:
# tracks

> If the track.artist column had been a dictionary of dictionaries, json_normalize would have handled it. However, it doesn't handle a list of dictionaries.

In [12]:
# type(tracks)

In [13]:
# len(tracks)

In [14]:
# tracks['track.artists'][0]

> If we want to slice the column, we have to do it this way:

In [15]:
# json_normalize(tracks['track.artists'][0])

> We´d like to extract the information in track.artist. Tracks is a pandas dataframe with 9925 rows, same lenght as the Spotify list. Column track.artist values contain a list with a dictionary. And we want to transform that list to a dictionary so that json_normalize recognises it and handles it.

In [16]:
# def list_to_dict(x):
#     return {i: x[i] for i in range(len(x))}

In [17]:
# for i in range(3230, 9925):
#     print('number ', i ,': ', len(tracks['track.artists'][i]))   

In [18]:
# json_normalize(tracks['track.artists'][3229])

> It looks like there is a float number in item 3229. I can´t even print that item. So I´ll skip that item in the function.
>
> I found a chain function here: https://stackoverflow.com/questions/14099872/concatenating-two-range-function-results

In [19]:
# from itertools import chain

In [20]:
# concatenated = list(chain(range(3229), range(3230, 9925)))

In [21]:
# def list_to_dict(x):   
#     return {i: x[i] for i in concatenated}

In [22]:
# type(concatenated[0])

In [23]:
# for i in concatenated:
#     print('number ', len(tracks['artist_dict'][i]))   

> For some reason, the function doesn´t work, but the iteration does work. I´ll do it with isinstance.

In [24]:
# def list_to_dict(x):
#     for i in concatenated:
#         return{i: x[i]}

In [25]:
# def list_to_dict(x):
#     result = {}
#     for i in len(tracks['track.artists']):
#         if not isinstance(x[i], float):
#             result[i] = x[i]
#     return result

In [26]:
def list_to_dict(x):
    if isinstance(x, list):
        return {i: x[i] for i in range(len(x))}
    else:
        return {}

In [27]:
tracks['artist_dict'] = tracks['track.artists'].apply(list_to_dict)

In [28]:
# tracks.shape

In [29]:
# tracks['artist_dict'][0]

> Now, we´d like to create a df. Every item of the artist_dict column will be a column in the new data frame. But every item has this structure: {0: {key:value, key:value, ...}}; so the function reads it as if 0 was the column, and we want it to be the row. So we´ll transpose it. It works the same way as the orient='index' seen in class.

In [30]:
def create_dict(row):
    df = pd.DataFrame.from_dict(row['artist_dict']).T
    df['song_id'] = row['track.id']
    return df

In [32]:
tracks['artists_dfs'] = tracks.apply(create_dict, axis=1)

In [33]:
# tracks['artists_dfs'][0]

> But we'd like to have those df in the main df. So, we will create a new one, empty, and concatenate each of the dfs. Later, we´ll merge it.

In [34]:
artist_df = pd.DataFrame(columns=['external_urls', 'href', 'id', 'name', 'type', 'uri'])
# artist_df

In [35]:
for df in tracks['artists_dfs']:
    artist_df = pd.concat([artist_df, df], axis=0)

In [36]:
# artist_df.head(5)

In [37]:
# artist_df.shape

In [38]:
# artist_df['id'].nunique()

In [39]:
# artist_df['name'].nunique()

In [40]:
# tracks['track.id'].nunique()

In [41]:
# artist_df['song_id']

In [42]:
# artist_df['song_id'].count()

In [43]:
# artist_df['song_id'].nunique()

In [44]:
# tracks['track.id'].nunique()

In [45]:
# artist_df.groupby(['song_id'])['name'].agg('count')

In [46]:
# tracks['track.name']

> In tracks, one row is created per name of song. But in artist_df, there are 12184 of wich only 9904 are unique, like on tracks. There can be more than one artist per song, so I want to keep all the information.

In [47]:
df_merged = pd.merge(left=tracks,
                    right=artist_df,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')

In [48]:
df_final = df_merged[['track.name', 'name', 'song_id']]

In [49]:
# df_final

In [50]:
# df_final.drop_duplicates()

In [51]:
# df_final.isna().sum()

# 2. Get the Features

https://developer.spotify.com/documentation/web-api/reference/get-audio-features

In [52]:
chunks = [(i, i+100) for i in range(0, len(df_final), 100)]

audio_features_list = []

for chunk in chunks:
    id_list100 = df_final['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)

    len(audio_features_list)

In [53]:
audio_features_df = json_normalize(audio_features_list)

In [54]:
# audio_features_df

In [55]:
df_ft = pd.merge(left=df_final,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')

In [56]:
# df_ft

In [57]:
df_ft.drop_duplicates(inplace=True)

> And this would be the final dataframe

In [58]:
df_ft

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Firework,Katy Perry,4lCv7b86sLynZbXhfScfm2,0.638,0.826,8,-4.968,1,0.0479,0.1390,0.000000,0.0803,0.649,124.072,audio_features,4lCv7b86sLynZbXhfScfm2,spotify:track:4lCv7b86sLynZbXhfScfm2,https://api.spotify.com/v1/tracks/4lCv7b86sLyn...,https://api.spotify.com/v1/audio-analysis/4lCv...,227880,4
1,All We Are,OneRepublic,1Jx69b09LKTuBQxkEiFfVX,0.397,0.817,2,-5.495,1,0.0420,0.0966,0.000008,0.3160,0.416,158.004,audio_features,1Jx69b09LKTuBQxkEiFfVX,spotify:track:1Jx69b09LKTuBQxkEiFfVX,https://api.spotify.com/v1/tracks/1Jx69b09LKTu...,https://api.spotify.com/v1/audio-analysis/1Jx6...,266227,4
2,Wake Up Alone,Amy Winehouse,4u83mwF5tUuWlXS86UOXdu,0.384,0.527,11,-5.294,0,0.0269,0.5420,0.000000,0.1560,0.222,100.730,audio_features,4u83mwF5tUuWlXS86UOXdu,spotify:track:4u83mwF5tUuWlXS86UOXdu,https://api.spotify.com/v1/tracks/4u83mwF5tUuW...,https://api.spotify.com/v1/audio-analysis/4u83...,221200,3
3,The Man Who Can't Be Moved,The Script,4Musyaro0NM5Awx8b5c627,0.608,0.629,10,-5.024,1,0.0264,0.4230,0.000000,0.0978,0.325,99.953,audio_features,4Musyaro0NM5Awx8b5c627,spotify:track:4Musyaro0NM5Awx8b5c627,https://api.spotify.com/v1/tracks/4Musyaro0NM5...,https://api.spotify.com/v1/audio-analysis/4Mus...,241467,4
4,Rolling in the Deep,Adele,1CkvWZme3pRgbzaxZnTl5X,0.729,0.756,8,-5.119,1,0.0294,0.1310,0.000000,0.0527,0.522,104.945,audio_features,1CkvWZme3pRgbzaxZnTl5X,spotify:track:1CkvWZme3pRgbzaxZnTl5X,https://api.spotify.com/v1/tracks/1CkvWZme3pRg...,https://api.spotify.com/v1/audio-analysis/1Ckv...,228293,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,I AM WOMAN,Emmy Meli,3nOz1U41SZZ0N3fuUWr9nb,0.646,0.471,10,-9.247,1,0.1570,0.1220,0.000000,0.1250,0.455,170.084,audio_features,3nOz1U41SZZ0N3fuUWr9nb,spotify:track:3nOz1U41SZZ0N3fuUWr9nb,https://api.spotify.com/v1/tracks/3nOz1U41SZZ0...,https://api.spotify.com/v1/audio-analysis/3nOz...,232813,4
21154,The Girl,City and Colour,1IFRVS4t1olI0XG9RBWdKH,0.404,0.542,10,-7.578,1,0.0349,0.8370,0.000072,0.1370,0.183,109.166,audio_features,1IFRVS4t1olI0XG9RBWdKH,spotify:track:1IFRVS4t1olI0XG9RBWdKH,https://api.spotify.com/v1/tracks/1IFRVS4t1olI...,https://api.spotify.com/v1/audio-analysis/1IFR...,360107,4
21155,The Funeral,Band of Horses,5lRzWDEe7UuedU2QPsFg0K,0.319,0.757,0,-6.117,0,0.0432,0.0160,0.000000,0.1020,0.101,122.465,audio_features,5lRzWDEe7UuedU2QPsFg0K,spotify:track:5lRzWDEe7UuedU2QPsFg0K,https://api.spotify.com/v1/tracks/5lRzWDEe7Uue...,https://api.spotify.com/v1/audio-analysis/5lRz...,322173,4
21156,Hallucinogenics,Matt Maeson,6bLU8e0LGyztE9iD5DWBQ1,0.549,0.653,0,-7.470,1,0.0503,0.4570,0.000000,0.0969,0.287,121.348,audio_features,6bLU8e0LGyztE9iD5DWBQ1,spotify:track:6bLU8e0LGyztE9iD5DWBQ1,https://api.spotify.com/v1/tracks/6bLU8e0LGyzt...,https://api.spotify.com/v1/audio-analysis/6bLU...,187173,4


In [59]:
df_ft.to_csv('features.csv', index=False)

### Now trying to Extract from the API

The lab is solved, but I want to try to extract it in other way.

In [69]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}

for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [ ]:
url = "https://accounts.spotify.com/api/token"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

data = {
    "grant_type": 'client_credentials',
    "client_id": secrets_dict['clientid'],
    "client_secret": secrets_dict['clientsecret']
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print("Access Token:", access_token)
else:
    print("Error:", response.status_code, response.text)


In [72]:
token = secrets_dict['token']

> Here I´m going to try with a song, following instructions in:
>
> https://developer.spotify.com/documentation/web-api/tutorials/getting-started#request-artist-data

In [78]:
url = 'https://api.spotify.com/v1/audio-features/11dFghVXANMlKmJXsNCbNl'
headers = {
    'Authorization': f'Bearer {token}'
}

res = requests.get(url, headers=headers)

if res.status_code == 200:
    data = res.json()
    # print(data)
else:
    print(f"Error: {res.status_code} - {res.text}")

In [80]:
df = pd.DataFrame([data])

In [94]:
# df

> Now I´m going to extract the features, following instructions in:
>
> https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

In [87]:
songs_ids = []
for i in df_final['song_id']:
    songs_ids.append(i)

In [91]:
url_list = []
for i in range(len(songs_ids)):
    url_list.append('https://api.spotify.com/v1/audio-features/'+ songs_ids[i])

In [92]:
url_list[0]

'https://api.spotify.com/v1/audio-features/4lCv7b86sLynZbXhfScfm2'

In [ ]:
df = pd.DataFrame()

for i in range(len(url_list)):
    url = url_list[i]
    headers = {
        'Authorization': f'Bearer {token}'
    }

    res = requests.get(url, headers=headers)

    if res.status_code == 200:
        data = res.json()
    else:
        print(f"Error: {res.status_code} - {res.text}")
     
    df = df.append(data, ignore_index=True)
    
    sleep(randint(1,3000)/1000)

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel

...

C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


Error: 401 - {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}


C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


Error: 401 - {
  "error": {
    "status": 401,
    "message": "The access token expired"
  }
}


C:\Users\PC\AppData\Local\Temp\ipykernel_15140\1181795267.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


> Will continue